<a href="https://colab.research.google.com/github/gabrielblins/palestra_gdg/blob/main/notebooks/GoogleExtended_OpenLLMLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integração Falcon 4bit - Langchain

Vamos ao que interessa.

### Software base
Vamos instalar o langchain e algumas outras coisas

In [17]:
!pip install -U -q langchain unstructured pinecone-client openai tiktoken python-dotenv sentence_transformers InstructorEmbedding transformers accelerate huggingface_hub faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 55.9 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv

load_dotenv()

False

# Integração com Pinecone

Para realizar uma integração completa usando exclusivamente o Maritaca, a API precisaria de um endpoint para captura dos embeddings, mas isso ainda não está disponível, então, a seguir iremos utilizar a OpenAI para criar as representações dos textos, enviar para o Pinecone e usar o Maritaca no final do fluxo. Funciona assim:

1. Capturamos os textos que queremos trabalhar, neste caso, a Portaria de Consolidação N 1 do Ministério da Saúde.

2. Vamos quebrar o texto em pequenas partes.

3. Interagir com a OpenAI e capturar os embeddings dos trechos criados.

4. Enviar esses trechos para o Pinecone.

Perceba que os passos acima são executados uma só vez ou sempre que houver necessidade de atualizar a estrutura. O resultado é uma base com diversos vetores que representam cada um dos trechos que criamos. O Pinecone, ou qualquer banco do mesmo tipo, viabiliza que sejam executadas consultas por similaridade. Dessa forma:

1. Agora que temos a base podemos enviar uma pergunta ou um texto qualquer, mas temos que criar a representação da pergunta e neste momento iremos chamar novamente a OpenAI para captura dos embeddings.

2. Em seguida, a Langchain interage com o Pinecone e pede para retornar os N vetores que mais se parecem com a pergunta/texto que enviamos.

Deste ponto em diante, já temos acesso aos N trechos semelhantes e, portanto, são os que possívelmente respondem a nossa pergunta. Vamos usar eles como contexto do prompt final com Maritaca.

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://arxiv.org/pdf/2304.07880.pdf")

In [4]:
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
print (f'Você tem {len(data)} documento(s) na base')
print (f'Há {len(data[0].page_content)} caracteres no documento')

Você tem 1 documento(s) na base
Há 65146 caracteres no documento


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100, separators=['\n', ' ', ''])
texts = text_splitter.split_documents(data)

In [9]:
print (f'Agora há {len(texts)} documentos')

Agora há 192 documentos


### Configuração da Embeddings e Pinecone

Recomendo ler como o Pinecone funciona, há muita documentação pela internet.

O código a seguir realiza a integração entre Pinecone e OpenAI e para isso você vai precisar de chaves. Para manter as que uso de forma privada, criei um arquivo .env com o seguinte formato:

```
    OPENAI_API_KEY=key
    PINECONE_API_KEY=key
    PINECONE_API_ENV=key
    PINECONE_INDEX=key
```

Chamei de .env e coloquei na raiz do colab. Você pode fazer o mesmo para testar este notebook.

In [10]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

import os

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [11]:
#OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
#PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
#PINECONE_API_ENV = os.environ['PINECONE_API_ENV']
#INDEX_NAME = os.environ['PINECONE_INDEX']

In [12]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large",
                                           model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [13]:
# initialize pinecone
#pinecone.init(
#    api_key=PINECONE_API_KEY,
#    environment=PINECONE_API_ENV
#)
#index_name = INDEX_NAME

In [14]:
# Use o código a seguir para criar a base
#docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name, namespace='compilers_book_os')

# Use o código a seguir se já tiver um índice criado
# docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings, namespace='compilers_book_os')

In [18]:
import faiss
from langchain.vectorstores import FAISS
import pickle

def store_embeddings(docs, embeddings, sotre_name, path):

    vectorStore = FAISS.from_documents(docs, embeddings)

    with open(f"{path}/faiss_{sotre_name}.pkl", "wb") as f:
        pickle.dump(vectorStore, f)

def load_embeddings(sotre_name, path):
    with open(f"{path}/faiss_{sotre_name}.pkl", "rb") as f:
        VectorStore = pickle.load(f)
    return VectorStore


In [19]:
docsearch = FAISS.from_texts([t.page_content for t in texts], embeddings)

### Base construída

Agora temos nossa base vetorizada! Nosso PDF contendo os dados da Portaria de Consolidação já está no Pinecone, vamos testar!

A variável query vai armazenar o texto que será usado para a pesquisa por similaridade. Note que ao criar o objeto docsearch a variável embeddings foi passada como parâmetro. Ela será usada para criar uma representação da query e comparar com os dados contidos no Pinecone. Até o momento, a Maritaca não tem essa opção de criar embeddings, então vamos usar a OpenAI mesmo.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 MB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=1024,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
query = "What are the Sabiá models?" # faça uma pergunta ou escreva um texto.
docs = docsearch.similarity_search(query)

### Integração final

Agora vamos usar os textos vindos do Pinecone e pedir para nosso LLM, o MariTalk, criar uma resposta adequada.

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate

PROMPT = """
Use the following text snippets to answer the question at the end. If you didn't know the answer, just say you don't know, don't try to create an answer.

{context}

Question: {question}
Answer:
"""

chain_prompt = PromptTemplate(input_variables=["context", "question"], template=PROMPT)

# note a variável llm sendo passada abaixo, ela é o Falcon7b quantizado
chain = load_qa_chain(llm, chain_type="stuff", verbose=True, prompt=chain_prompt)

In [ ]:
query = "What are the Sabiá models?" # faça uma pergunta ou escreva um texto.

# no exemplo abaixo reduzimos o número de documentos para caber na janela do
# Falcon
docs = docsearch.similarity_search(query, k=3)
chain.run(input_documents=docs, question=query)